## Project Runner

### I. Import packages

### II. Data Cleanning and Setup

In [5]:
# Copy code from cleaner.ipynb 
# OR import clean .csv produced from cleaner

# Split data into trainning-validation-test

### III. Trainning

In [7]:
# Implement ML classifiers

# May include experiments of tunning hyperparameters

### IV. Testing and Graph